# Eat Safe, Love

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

## Part 1: Database and Jupyter Notebook Set Up

In [37]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [38]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [39]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'class_DB', 'config', 'garden_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [40]:
# assign the uk_food database to a variable name
uk_food_db = mongo['uk_food']


In [41]:
# review the collections in our new database
print(uk_food_db.list_collection_names())



['establishments']


In [34]:
# Drop code in case of errors

# uk_food_db.establishments.drop()
# uk_food_db.client.drop_database(uk_food_db)



In [42]:
# review a document in the establishments collection
pprint(uk_food_db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65a4b8f997313ddb00d70609'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [43]:
# assign the collection to a variable
establishments = uk_food_db['establishments']


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [44]:
# Create a dictionary for the new restaurant data
new_customer = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [45]:
# Check that the new restaurant was inserted
establishments.insert_one(new_customer)

InsertOneResult(ObjectId('65a4b914b9f779d2c0d1042a'), acknowledged=True)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [46]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': {'$regex': 'Restaurant/Cafe/Canteen'}}
fields = {'BusinessType': 1, 'BusinessTypeID': 1}
results = establishments.find(query)

# Pretty print the first two results
pprint( establishments.find_one(query, fields))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65a4b8f997313ddb00d7060a')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [47]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set':
     {'BusinessTypeID': 1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [48]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)

pprint(establishments.find_one(query))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65a4b914b9f779d2c0d1042a'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [49]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
# results = list(establishments.aggregate(query))
establishments.count_documents((query))


994

In [50]:
# Delete all documents where LocalAuthorityName is "Dover"
uk_food_db.establishments.delete_many(
    {'LocalAuthorityName': 'Dover'}
)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [51]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
establishments.count_documents((query))

0

In [52]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('65a4b8f997313ddb00d708ec'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [53]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {}, [{'$set': 
          {'geocode.latitude' : {'$toDouble': '$geocode.latitude'},
           'geocode.longitude' : {'$toDouble': '$geocode.longitude'}}}])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [54]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [{'$set':{ "RatingValue" : None}}])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [56]:
# Change the data type from String to Integer for RatingValue

# Issue with the code updating - it was updating the rating field with {'$toInt': '$RatingValue'} and not converting to an integer
# establishments.update_many({}, {'$set':
#                                  {'RatingValue' : {'$toInt': '$RatingValue'}}})

establishments.update_many({}, [{'$addFields': {'RatingValue': {'$toInt': '$RatingValue'}}}])


UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [57]:
# Check that the coordinates and rating value are now numbers
establishments.find_one()

{'_id': ObjectId('65a4b8f997313ddb00d708ec'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00